#  Ejercicio: Un sencillo detector de idioma en documentos

Vamos a realizar otro ejercicio de modelización, ahora con un caso de uso de reconocimiento de patrones: distinguir si un documento está escrito en inglés o en español.

(No vamos a usar técnicas de NLP "de verdad", sino conceptos básicos para seguir ilustrando la filosofía de la modelización. Más adelante tenéis un curso de NLP). 

Recordemos que con **modelización** nos referimos a adoptar una metodología para tomar decisiones automáticas basadas en datos. 

Que un modelo tiene: 

* Unos parámetros
* Un proceso para "ajustar" los parámetros
* Un proceso para aplicar la decisión a nuevos datos

Y que:

* En muchos casos el modelo es un resumen de los datos. 
* Si los datos usados para ajustar cambian, la decisión debería cambiar (esto es bueno!)
* Debería mejorar conforme más datos se usan para ajustar
* Está basado en suposiciones!! Un modelo es como mínimo igual de bueno que las suposiciones que se han realizado para llevarlo a cabo. 

Este ejercicio nos permitirá consolidar todos esos conceptos. 

## Ejercicio

¿Cómo diseñarías una función simple que identificase si una cadena de texto está en inglés o en español? (Esto es un problema habitual en redes sociales o gestión documental, entre otros). 

Vamos a usar un pequeño dataset derivado de https://github.com/FerreroJeremy/Cross-Language-Dataset.git

In [21]:
#!tree --filelimit 5 dataset-lang/

dataset-lang/
├── files [200 entries exceeds filelimit, not opening dir]
├── test-en.txt
├── test-es.txt
├── training-en.txt
└── training-es.txt

1 directory, 4 files


Tenemos un directorio "files" con los documentos. Y unos ficheros *.txt de listas. 

Estos ficheros sirven para 2 cosas: especifican qué documentos están en inglés y cuáles en español (veremos el concepto de "etiqueta"). 

También los hemos dividido a mano en los documentos que usaremos para ajustar el modelo (training) y los que después usaremos para simular cómo de bien o mal habría funcionado (test). Esta partición es una práctica habitual en modelización, sobre todo en machine learning. 

Imaginemos cómo se podría detectar un idioma en un documento mediante algún criterio práctico y sencillo. 

In [1]:
def detect_lang(sentence):
     
    votes_english = 0 
    votes_spanish = 0
    
    if "ñ" in sentence:
        votes_spanish += 100
            
    for s in sentence.split(" "):
        
        if s in ['the', 'this', 'in', 'where', 'take']:
            votes_english += 1
        if s in ['el', 'la', 'este', 'esta', 'en', 'que']:
            
            votes_spanish += 1
                    
    if votes_spanish == votes_english:
        return None
    elif votes_spanish > votes_english:
        return 'es'
    else:
        return 'en'

In [ ]:
!head -n 1 dataset-lang/training-es.txt

Primero, vamos a probar esta en un fichero. Aquí una función que convierte el fichero a cadena y le aplica el criterio que hemos programado. 

In [2]:
import sys
import re

FILE_ES = "dataset-lang/files/ep-00-01-17-es.txt"

def file2str(file):

    with open(file,  encoding='utf-8', errors='ignore') as f:
        lines = f.readlines()
        str = ""
        for s in lines:
            str += re.sub( '\s+', ' ', s.lower().replace("\n", "") ).strip()
        #print("Read", file, "\nnchars =", len(str), file=sys.stderr)
    
    return str
        
detect_lang(file2str(FILE_ES))

'es'

In [3]:
detect_lang('Here is the document')

'en'

Sí, hemos tomado una decisión automáticamente, pero está basada en conocimiento a priori y manualmente codificado. 

Vamos a ver ahora una estrategia basada en datos: 

In [4]:
import numpy as np

PREFIX = "dataset-lang/files/"

def collect_words(lang):

    # lang is "es" or "en"
    
    words = dict()
    
    # Collect words into dict
    filelist = "dataset-lang/training-" + lang + ".txt"
    with open(filelist) as f:
        
        lines = f.readlines()
        for l in lines:
            realfile = PREFIX + l.replace("\n", "")
            str_ = file2str(realfile)
            words_in_file  = str_.split()
            for w in words_in_file:
                if w in words:
                    words[w] += 1
                else:
                    words[w] = 1
            
    print("nwords = ", len(words), file=sys.stderr)
    
    # Finished collecting, keep top 50 words
    values = -np.array(list(words.values()))
    keys = np.array(list(words.keys()))

    selected = np.argpartition(values, 50)[:50]
    return keys[selected], values[selected]
        
    
def vote(sentence, dict_es, dict_en):
    
    score_en = 0
    score_es = 0
    
    for w in sentence.split(" "):
        if w in dict_es:
            score_es +=1
        if w in dict_en:
            score_en += 1
            
    return score_es, score_en


In [5]:
%%time
w_es, _ = collect_words("es")
w_en, _ = collect_words("en")

nwords =  248024
nwords =  180751


Wall time: 1min 2s


Podemos ver la salida del detector en un par de frases de ejemplo: 

In [6]:
vote("a ver en que idioma detecta esta frase", w_es, w_en)

(4, 1)

In [7]:
vote("now let's try a text in english", w_es, w_en)

(1, 2)

Aunque lo que se suele hacer es medir el porcentaje de acierto en otro conjunto de muestras, al que se suele llamar "conjunto de test". 

In [8]:
TEST_LIST_ES = 'dataset-lang/test-es.txt'

n_es_correct = 0
n = 0
with open(TEST_LIST_ES) as f:
        
    lines = f.readlines()
    for l in lines:
        realfile = PREFIX + l.replace("\n", "")
        str_ = file2str(realfile)
        result = vote(str_, w_es, w_en)
        if result[0] > result[1]:
            n_es_correct += 1
        n += 1

In [9]:
n_es_correct

20

Aunque es un ejemplo sencillo, esta solución es más elegante que las reglas anteriores, porque no hemos diseñado reglas por prueba y error. Al contrario, hemos "ajustado" el modelo "enseñándole" cómo son documentos en inglés y en español. La metodología consiste en detectar las palabras más frecuentes durante el ajuste y para aplicar a nuevos documentos, se realiza un proceso de votación. 

Podemos decir que el ajuste es la fase en la que "enseñamos" al sistema lo que tiene que saber para hacer la tarea. 

Además, vamos a evaluar los puntos de antes: 

* **¿El modelo es un resumen de los datos?** Sí. Fijémonos que cuando ya está "ajustado", sólo necesitamos las listas de palabras, no los documentos enteros. 

* **¿Si los datos cambian, la decisión cambia?** Sí.: Si en lugar de artículos fuesen tweets. Fijemonos que exactamente el mismo sistema podría usarse para distinguir inglés de francés. Ojo: necesitaremos datos de documentos en francés. 

* **¿Mejora con más datos?**: Sí. Cuantos más datos, más fiables las palabras más frecuentes.  

* **¿Está basado en suposiciones?** Sí. (Por ejemplo, que los documentos de aprendizaje no tienen mezcla de idiomas, que están bien "tokenizados", que son suficientemente largos, etc). 



Aquí ya podemos empezar a adelantar terminología de metodología general de modelización, que se explican durante clase:

* Variables (características, atributos, variables dependientes, entradas). 
* "Fit": Ajustar el modelo (entrenar, aprender)
* "Predict": Aplicar el modelo (testear, predecir, inferencia)
* Conjunto de entrenamiento y conjunto de test
* Representación
* Etiquetas ("targets", variable dependiente, salidas) 
* Hiperparámetros
* Evaluación
* (No tratado) Función objetivo

<br>
**Take-home messages**

Hemos visto un ejemplo (muy simple, hello world!) de modelos. Lo importante a retener es que

* De nuevo, un modelo es una metodología para tomar decisiones automáticas a partir de datos (y suposiciones). Dados unos datos (training set), "aprende" la información necesaria para realizar deducciones (a veces se habla de predicciones) sobre nuevos datos. Terminología se habla de ajustar un modelo y aplicar el modelo. 

* Si se hace bien, es una solución más flexible que programar los criterios a mano. 

* Sin datos no hay modelo. Cuanto mejor sean los datos, mejor será el modelo. 

* Hay que verificar las suposiciones continuamente.

* Se han aprendido nuevos términos: conjunto de entrenamiento, de test, representación, etiquetas, hiperparámetros. 


<br>
<div style="background-color:orange;border-radius:20px;padding:20px">
Atención!!! A veces, sobre todo en el contexto de modelos de machine learning, se habla de estos modelos como "sistemas a los que no hay que programar, ya que se aprenden a programar ellos solos a partir de los datos". Es una afirmación peligrosa si se omiten los requisitos: no es que no se programen, sino que no se programan unas reglas manuales. Lo que se programa es la capacidad del sistema para ajustar y aplicar un modelo en base a unos datos. También tiene que quedar claro que los datos son toda la información necesaria para ajustar el modelo. Por ejemplo, en este caso: los documentos y sus etiquetas. Los documentos sin etiquetas son insuficientes para que un sistema aprenda.   
     
    </b>
</div>
 
<br>
Ver por ejemplo el post de [software 2.0](https://medium.com/@karpathy/software-2-0-a64152b37c35) (es controversial). 
<br><br>

<div style="background-color:orange;border-radius:20px;padding:20px">
<p>Atención!!!El ejemplo es muy sencillo a propósito. Aunque funciona relativamente bien, los profesionales que necesitan detectar idiomas usan métodos más complejos como clasificadores con redes neuronales. 
<p>Aunque también es cierto que los profesionales de data science aprenden con el tiempo que si un método sencillo es good enough, adelante. 
    </b>
</div>
